In [ ]:
print ("importing arcpy")
import arcpy
import sys
from arcpy import env
from arcpy.sa import *
arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True
arcpy.env.transferDomains = True

In [ ]:
path = r"N:\Projects\CA_FWA\projects\UnimparedFlows"
path_data = path + "\\archive\\CA_data"
path_tables = path + "\\tables"
gages_comid = path_tables + "\\Gage_Comid_All.csv"
gages_data = path_tables + "\\Gages_Data_All.csv"

gage_actual = path_data + "\\AllGages_Observed_Monthlies\\All_Mn_Q.csv"

region_eco_list = [
    ["COAST", ["6.2.11","7.1.8"]],
    ["MNTS", ["6.2.7","6.2.8","6.2.12"]],
    ["XERIC", ["10.1.3","10.1.5","10.2.1","10.2.2","11.1.1","11.1.2","11.1.3"]]   
]

In [ ]:
# Open file with COMID for gages only and covert to a list (note that there is a carriage return at the end of each COMID)

f = open(gages_comid, 'r')
gages = f.readlines()
f.close()

In [ ]:
# Build a list of the gages and the COMID
gage_pair_list = []
for gage in gages:
    gage_pair = str(gage.strip()).split(",")
    gage_pair_list.append(gage_pair)

#Build a list of gages
gage_list = []
for gage_pair in gage_pair_list:
    gage_list.append(gage_pair[0])

#Build a list of COMID
comid_list = []
for gage_pair in gage_pair_list:
    comid_list.append(gage_pair[1])

# Build a dictionary that will return a gage or list of gages for every comid 
gage_pair_dict = {}
for gage_pair in gage_pair_list:
    if gage_pair[1] in gage_pair_dict:
        print("duplicate record for " + str(gage_pair[1]))
        templist = gage_pair_dict[gage_pair[1]]
        templist.append(gage_pair[0])
        gage_pair_dict[gage_pair[1]] = templist
    else:
        gage_pair_dict[gage_pair[1]] = [gage_pair[0]]

In [ ]:
# Make table for output
f_out = open(gages_data, 'w')
f_out.write('"Gage_ID","Date","Type","Value"\n')

In [ ]:
# Extract gage data
f_out = open(gages_data, 'a')
a = open(gage_actual, "r")
alines = a.readlines()

for line in alines:
    line_list = line.split(",")
    line_gage_ID = line_list[0][1:-1]
    if line_gage_ID in gage_list:
        date = line_list[2] + "/1/" + line_list[1]
        f_out.write(line_gage_ID + "," + date + ",Actual mean Q," + line_list[4] + "\n")
        
f_out.close()

In [ ]:
# Export to data with matching COMID to new table

f_out = open(gages_data, 'a')

for region_tuple in region_eco_list:
    region = region_tuple[0]
    eco_list = region_tuple[1]
    
    for eco in eco_list:

        months = list(range(2,13))
        #months = list(range(1,13))

        for month in months:
            if month < 10:
                month_str = "0" + str(month)
            else:
                month_str = str(month)
            table = path_data + "\\" + region + "\\Mean\\" + eco + "_" + month_str + "_Pred.csv"
            print("Reading data for " + table)
            t = open(table, 'r')
            lines = t.readlines()

            for line in lines:
                line_list = line.split(",")
                if line_list[0] in comid_list:
                    for gage_id in gage_pair_dict[line_list[0]]: # this will duplicate the data for the comids with multiple gages
                        date = str(month) + "/1/" + line_list[2]
                        f_out.write(gage_id + "," + date + ",P10 mean Q," + str(line_list[7]) + "\n")
                        f_out.write(gage_id + "," + date + ",P90 mean Q," + str(line_list[9]) + "\n")
                        f_out.write(gage_id + "," + date + ",Estimated mean Q," + str(line_list[10]) + "\n")

        t.close()
f_out.close()
